In [1]:
!pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV  #finding best model
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')




ModuleNotFoundError: No module named 'torch'

In [ ]:
df=pd.read_csv('/content/draft_50000.csv')
#df.head()
df['text'] = df['text'].str.lower().str.replace('[^\w\s]', '', regex=True)
df.dropna(subset=['text', 'class'], inplace=True)
#df.head()

In [ ]:
import nltk # Import the nltk module first
nltk.download('punkt')
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english') # Use words() instead of word()
stopwords.extend(['.',',','!','?',';',':'])
#Remove stopwords from a given text
def remove_stopword(txt):
  # Use word_tokenize instead of word_tokenizer
  tokens=nltk.word_tokenize(txt)
  new_tokens=[token for token in tokens if token.lower() not in stopwords]
  return ' '.join(new_tokens)
df['text']  =df['text'].apply(remove_stopword)
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,text,class
0,51297,get crush guy definitely way old laugh literal...,0
1,24705,go july 2018i hope 8 month enough change mind,1
2,185969,want live anymore 23 right thinking end life p...,1
3,201675,every time get period want kill already depres...,1
4,52701,story incomplete similar story op case dad spe...,1


In [ ]:
# For BERT:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

## Want DistilBERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenized = df["text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
display(tokenized)

Token indices sequence length is longer than the specified maximum sequence length for this model (704 > 512). Running this sequence through the model will result in indexing errors


0        [101, 2131, 10188, 3124, 5791, 2126, 2214, 475...
1        [101, 2175, 2251, 2760, 2072, 3246, 1022, 3204...
2        [101, 2215, 2444, 4902, 2603, 2157, 3241, 2203...
3        [101, 2296, 2051, 2131, 2558, 2215, 3102, 2525...
4        [101, 2466, 12958, 2714, 2466, 6728, 2553, 361...
                               ...                        
49995    [101, 2082, 17704, 11891, 2812, 2082, 2202, 20...
49996    [101, 10188, 6719, 2425, 2196, 2191, 8257, 305...
49997    [101, 4064, 8322, 2102, 2514, 2802, 2627, 1016...
49998    [101, 2425, 2242, 3407, 4148, 2516, 2428, 2360...
49999    [101, 2717, 3521, 2467, 3961, 2540, 2467, 2293...
Name: text, Length: 49989, dtype: object

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
np.array(padded).shape

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

In [ ]:
!pip install torch

import torch

max_length = 512

input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

# Truncate input_ids and attention_mask
input_ids = input_ids[:, :max_length]
attention_mask = attention_mask[:, :max_length]

# Proceed with the model inference
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
'''max_length = 512

input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

# Truncate input_ids and attention_mask
input_ids = input_ids[:, :max_length]
attention_mask = attention_mask[:, :max_length]

# Proceed with the model inference
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
'''

In [ ]:

print(attention_mask.size())

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df["class"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(text, class)

In [ ]:

lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)

In [ ]:


import numpy as np
pred= lr_clf.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test,pred))